<a href="https://colab.research.google.com/github/dhsingh0815/ML/blob/main/Bayes/SpamIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

df=pd.read_csv('/content/spam_2.csv')
df.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
import string
df['v2']=df['v2'].map(lambda sentence: sentence.lower().translate(str.maketrans('', '', string.punctuation)))
df.head(5)

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [5]:
stop_words=nltk.corpus.stopwords.words('english')

#[ var for var in iterable if condition: statement]
def prepocessing(sentence):
  words=sentence.split()
  filtered_tokens=[word for word in words if word not in stop_words]
  return ' '.join(filtered_tokens)

In [6]:
df['Filtered_tokens']=df['v2'].map(prepocessing)
df.head(5)

,v1,v2,Filtered_tokens
0,ham,go until jurong point crazy available only in ...,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say so early hor u c already then say,u dun say early hor u c already say
4,ham,nah i dont think he goes to usf he lives aroun...,nah dont think goes usf lives around though


In [8]:
label ={'ham' :0 , 'spam':1}
df['label_encode']=df['v1'].map(label)
df.head(5)


,v1,v2,Filtered_tokens,label_encode
0,ham,go until jurong point crazy available only in ...,go jurong point crazy available bugis n great ...,0
1,ham,ok lar joking wif u oni,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,u dun say so early hor u c already then say,u dun say early hor u c already say,0
4,ham,nah i dont think he goes to usf he lives aroun...,nah dont think goes usf lives around though,0


In [9]:
df['label_encode'].value_counts()

,count
label_encode,
0,4825
1,747


We have imbalanced data , how to solve it as it is not numeric column

In [7]:

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

In [10]:
df_selected=df[['Filtered_tokens','label_encode']]
df_selected.head(5)

,Filtered_tokens,label_encode
0,go jurong point crazy available bugis n great ...,0
1,ok lar joking wif u oni,0
2,free entry 2 wkly comp win fa cup final tkts 2...,1
3,u dun say early hor u c already say,0
4,nah dont think goes usf lives around though,0


In [15]:
X=df_selected['Filtered_tokens']
y =df_selected['label_encode']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [14]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(sampling_strategy='minority')
# X_smote,y_smote=smote.fit_resample(X,y)

ValueError: could not convert string to float: 'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X_smote,y_smote,test_size=0.2)
# y_test.value_counts()